In [1]:
import pickle
import numpy as np
import neat
import datetime
import os
import math

from comunication_channel import AgentLogChannel

from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.base_env import ActionTuple
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel

In [2]:
def add_node(g, config, new_node_id=None, bias=None, response=None, aggregation=None, activation=None):
    if new_node_id is None:
        new_node_id = config.get_new_node_key(g.nodes)
    ng = g.create_node(config, new_node_id)
    g.nodes[new_node_id] = ng
    if bias is not None:
        ng.bias = bias
    if response is not None:
        ng.response = response
    if aggregation is not None:
        ng.aggregation = aggregation
    if activation is not None:
        ng.activation = activation
    return new_node_id, ng

In [3]:
import neat
from random import choice, gauss, random, uniform, randint
from neat.genome import DefaultGenome
from neat.genes import DefaultNodeGene, DefaultConnectionGene
from neat.config import ConfigParameter, write_pretty_params
from neat.genes import BaseGene
from neat.attributes import FloatAttribute, BoolAttribute, StringAttribute, BaseAttribute
from neat.activations import ActivationFunctionSet
from neat.aggregations import AggregationFunctionSet
from itertools import count

class FixedFloatAttribute(BaseAttribute):
    """
    Class for floating-point numeric attributes,
    such as the response of a node or the weight of a connection.
    """
    _config_items = {"init_mean": [float, None],
                     "init_stdev": [float, None],
                     "init_type": [str, 'gaussian'],
                     "replace_rate": [float, None],
                     "mutate_rate": [float, None],
                     "mutate_power": [float, None],
                     "max_value": [float, None],
                     "min_value": [float, None]}

    def clamp(self, value, config):
        min_value = getattr(config, self.min_value_name)
        max_value = getattr(config, self.max_value_name)
        return max(min(value, max_value), min_value)

    def init_value(self, config):
        mean = getattr(config, self.init_mean_name)
        stdev = getattr(config, self.init_stdev_name)
        init_type = getattr(config, self.init_type_name).lower()

        if ('gauss' in init_type) or ('normal' in init_type):
            return self.clamp(gauss(mean, stdev), config)

        if 'uniform' in init_type:
            min_value = max(getattr(config, self.min_value_name),
                            (mean - (2 * stdev)))
            max_value = min(getattr(config, self.max_value_name),
                            (mean + (2 * stdev)))
            return uniform(min_value, max_value)

        raise RuntimeError(f"Unknown init_type {getattr(config, self.init_type_name)!r} for {self.init_type_name!s}")

    def mutate_value(self, value, config):
        # mutate_rate is usually no lower than replace_rate, and frequently higher -
        # so put first for efficiency
        mutate_rate = getattr(config, self.mutate_rate_name)
        min_value = getattr(config, self.min_value_name)
        max_value = getattr(config, self.max_value_name)
        delta = choice([min_value / 100, max_value / 100])
        r = random()
        if r < mutate_rate:
            mutation = (2 * random() - 1) * delta
            return  self.clamp(value + mutation, config)
        
        r = random()
        if r < mutate_rate:
            mutate_power = getattr(config, self.mutate_power_name)
            mutation = (2 * random() - 1) * mutate_power
            return  self.clamp(value + mutation, config)
        
        replace_rate = getattr(config, self.replace_rate_name)

        r = random()
        if r < replace_rate:
            return self.init_value(config)

        return value

    def validate(self, config):
        min_value = getattr(config, self.min_value_name)
        max_value = getattr(config, self.max_value_name)
        if max_value < min_value:
            raise RuntimeError("Invalid min/max configuration for {self.name}")

class FixedStructureNodeGene(DefaultNodeGene):
    _gene_attributes = [
                        FloatAttribute('bias'),
                        FloatAttribute('response'),
                        StringAttribute('activation', options=''),
                        StringAttribute('aggregation', options='')]
    def __init__(self, key):
        super().__init__(key)
        DefaultNodeGene.__init__(self, key)
    # def distance(self, other, config):
    #     d = abs(self.bias - other.bias) + abs(self.response - other.response)
    #     if self.activation != other.activation:
    #         d += 1.0
    #     if self.aggregation != other.aggregation:
    #         d += 1.0
    #     return d * config.compatibility_weight_coefficient

class FixedStructureConnectionGene(DefaultConnectionGene):
    _gene_attributes = [FloatAttribute('weight'),
                        BoolAttribute('enabled')]
    def __init__(self, key):
        super().__init__(key)
        DefaultConnectionGene.__init__(self, key)
    # def mutate(self, config):
    #     # Mutate only weights
    #     a = self._gene_attributes[0]
    #     v = getattr(self, a.name)
    #     setattr(self, a.name, a.mutate_value(v, config))

    # def distance(self, other, config):
    #     d = abs(self.weight - other.weight)
    #     if self.enabled != other.enabled:
    #         d += 1.0
    #     return d * config.compatibility_weight_coefficient
    
class FixedStructureGenomeConfig(object):
    __params = [ConfigParameter('feed_forward', bool),
                ConfigParameter('num_inputs', int),
                ConfigParameter('num_outputs', int),
                ConfigParameter('num_hidden', int),
                ConfigParameter('num_hidden_layers', int), 
                ConfigParameter('initial_connection', str, 'full_direct'),
                ConfigParameter('conn_add_prob', float),
                ConfigParameter('conn_delete_prob', float),
                ConfigParameter('node_add_prob', float),
                ConfigParameter('node_delete_prob', float),
                ConfigParameter('compatibility_disjoint_coefficient', float),
                ConfigParameter('compatibility_weight_coefficient', float),
                ConfigParameter('structural_mutation_surer', str, 'default'), 
                ConfigParameter('single_structural_mutation', bool, 'false')]


    def __init__(self, params):
        # Create full set of available activation functions.
        self.activation_defs = ActivationFunctionSet()
        self.activation_options = params.get('activation_options', 'clamped').strip().split()
        self.aggregation_options = params.get('aggregation_options', 'sum').strip().split()
        self.aggregation_function_defs = AggregationFunctionSet()

        # Gather configuration data from the gene classes.
        self.node_gene_type = params['node_gene_type']
        self.__params += self.node_gene_type.get_config_params()
        self.connection_gene_type = params['connection_gene_type']
        self.__params += self.connection_gene_type.get_config_params()
        self.connection_fraction = None
        self.node_indexer = None
        self.structural_mutation_surer = 'false'

        # Use the configuration data to interpret the supplied parameters.
        for p in self.__params:
            setattr(self, p.name, p.interpret(params))

        # By convention, input pins have negative keys, and the output
        # pins have keys 0,1,...
        self.input_keys = [-i - 1 for i in range(self.num_inputs)]
        self.output_keys = [i for i in range(self.num_outputs)]

    def get_new_node_key(self, node_dict):
        if self.node_indexer is None:
            if node_dict:
                self.node_indexer = count(max(list(node_dict)) + 1)
            else:
                self.node_indexer = count(max(list(node_dict)) + 1)

        new_id = next(self.node_indexer)

        assert new_id not in node_dict

        return new_id
    
    def save(self, f):
        write_pretty_params(f, self, self.__params)

    def check_structural_mutation_surer(self):
        if self.structural_mutation_surer == 'true':
            return True
        elif self.structural_mutation_surer == 'false':
            return False
        elif self.structural_mutation_surer == 'default':
            return self.single_structural_mutation
        else:
            error_string = f"Invalid structural_mutation_surer {self.structural_mutation_surer!r}"
            raise RuntimeError(error_string)
        
class FixedStructureGenome(DefaultGenome):
    @classmethod
    def parse_config(cls, param_dict):
        param_dict['node_gene_type'] = FixedStructureNodeGene
        param_dict['connection_gene_type'] = FixedStructureConnectionGene
        return FixedStructureGenomeConfig(param_dict)
    
    @classmethod
    def write_config(cls, f, config):
        config.save(f)

    def __init__(self, key):
        self.key = key
        self.connections = {}
        self.nodes = {}

        # Fitness results.
        self.fitness = None

    def mutate(self, config):
        """ Mutates this genome. """

        if config.single_structural_mutation:
            div = max(1, (config.node_add_prob + config.node_delete_prob +
                            config.conn_add_prob + config.conn_delete_prob))
            r = random()
            if r < (config.node_add_prob / div):
                self.mutate_add_node(config)
            elif r < ((config.node_add_prob + config.node_delete_prob) / div):
                self.mutate_delete_node(config)
            elif r < ((config.node_add_prob + config.node_delete_prob +
                        config.conn_add_prob) / div):
                self.mutate_add_connection(config)
            elif r < ((config.node_add_prob + config.node_delete_prob +
                        config.conn_add_prob + config.conn_delete_prob) / div):
                self.mutate_delete_connection()
        else:
            if random() < config.node_add_prob:
                self.mutate_add_node(config)

            if random() < config.node_delete_prob:
                self.mutate_delete_node(config)

            if random() < config.conn_add_prob:
                self.mutate_add_connection(config)

            if random() < config.conn_delete_prob:
                self.mutate_delete_connection()

        # Mutate connection genes.
        for cg in self.connections.values():
            cg.mutate(config)

        # Mutate node genes (bias, response, etc.).
        for ng in self.nodes.values():
            ng.mutate(config)

    def configure_crossover(self, genome1, genome2, config):
        """ Configure a new genome by crossover from two parent genomes. """
        if genome1.fitness > genome2.fitness:
            parent1, parent2 = genome1, genome2
        else:
            parent1, parent2 = genome2, genome1

        # Inherit connection genes
        for key, cg1 in parent1.connections.items():
            cg2 = parent2.connections.get(key)
            if cg2 is None:
                # Excess or disjoint gene: copy from the fittest parent.
                self.connections[key] = cg1.copy()
            else:
                # Homologous gene: combine genes from both parents.
                self.connections[key] = cg1.crossover(cg2)
            self.connections[key].enabled = True

        # Inherit node genes
        parent1_set = parent1.nodes
        parent2_set = parent2.nodes

        for key, ng1 in parent1_set.items():
            ng2 = parent2_set.get(key)
            assert key not in self.nodes
            if ng2 is None:
                # Extra gene: copy from the fittest parent
                self.nodes[key] = ng1.copy()
            else:
                # Homologous gene: combine genes from both parents.
                self.nodes[key] = ng1.crossover(ng2)

    def distance(self, other, config):
        """
        Returns the genetic distance between this genome and the other. This distance value
        is used to compute genome compatibility for speciation.
        """

        # Compute node gene distance component.
        node_distance = 0.0
        if self.nodes or other.nodes:
            disjoint_nodes = 0
            for k2 in other.nodes:
                if k2 not in self.nodes:
                    disjoint_nodes += 1

            for k1, n1 in self.nodes.items():
                n2 = other.nodes.get(k1)
                if n2 is None:
                    disjoint_nodes += 1
                else:
                    # Homologous genes compute their own distance value.
                    node_distance += n1.distance(n2, config)

            max_nodes = max(len(self.nodes), len(other.nodes))
            node_distance = (node_distance +
                                (config.compatibility_disjoint_coefficient *
                                disjoint_nodes)) / max_nodes

        # Compute connection gene differences.
        connection_distance = 0.0
        if self.connections or other.connections:
            disjoint_connections = 0
            for k2 in other.connections:
                if k2 not in self.connections:
                    disjoint_connections += 1

            for k1, c1 in self.connections.items():
                c2 = other.connections.get(k1)
                if c2 is None:
                    disjoint_connections += 1
                else:
                    # Homologous genes compute their own distance value.
                    connection_distance += c1.distance(c2, config)

            max_conn = max(len(self.connections), len(other.connections))
            connection_distance = (connection_distance +
                                    (config.compatibility_disjoint_coefficient *
                                    disjoint_connections)) / max_conn

        distance = node_distance + connection_distance
        return distance
    
    def size(self):
        """Returns genome 'complexity', taken to be (number of nodes, number of enabled connections)"""
        num_enabled_connections = sum([1 for cg in self.connections.values() if cg.enabled is True])
        return len(self.nodes), num_enabled_connections

    def __str__(self):
        s = f"Key: {self.key}\nFitness: {self.fitness}\nNodes:"
        for k, ng in self.nodes.items():
            s += f"\n\t{k} {ng!s}"
        s += "\nConnections:"
        connections = list(self.connections.values())
        connections.sort()
        for c in connections:
            s += "\n\t" + str(c)
        return s

    def configure_new(self, config):
        # Create node genes for the input and output nodes
        for node_key in config.output_keys:
            self.nodes[node_key] = self.create_node(config, node_key)

        # Initialize list to keep track of nodes in the last layer (start with input layer)
        last_layer_nodes = config.input_keys

        # Add hidden nodes and layers
        # config.num_hidden now represents the number of neurons PER HIDDEN LAYER
        if config.num_hidden > 0 and config.num_hidden_layers > 0:
            for _ in range(config.num_hidden_layers):
                layer_node_ids = []
                for _ in range(config.num_hidden):  # Create the specified number of nodes for this layer
                    node_key = config.get_new_node_key(self.nodes)
                    self.nodes[node_key] = self.create_node(config, node_key)
                    layer_node_ids.append(node_key)
                
                # Connect all nodes from the last layer to the current layer's nodes
                for last_layer_node in last_layer_nodes:
                    for current_layer_node in layer_node_ids:
                        connection = self.create_connection(config, last_layer_node, current_layer_node)
                        self.connections[connection.key] = connection

                # Update last_layer_nodes to reflect the current layer for next iteration
                last_layer_nodes = layer_node_ids

            # Connect all nodes from the last hidden layer to the output nodes
            for hidden_node_id in last_layer_nodes:
                for output_node_id in config.output_keys:
                    connection = self.create_connection(config, hidden_node_id, output_node_id)
                    self.connections[connection.key] = connection
        else:
            # If there are no hidden layers, connect input nodes directly to output nodes
            for input_node_id in config.input_keys:
                for output_node_id in config.output_keys:
                    connection = self.create_connection(config, input_node_id, output_node_id)
                    self.connections[connection.key] = connection

    @staticmethod
    def create_node(config, node_id):
        node = FixedStructureNodeGene(node_id)
        node.init_attributes(config)
        return node

    @staticmethod
    def create_connection(config, input_id, output_id):
        connection = config.connection_gene_type((input_id, output_id))
        connection.init_attributes(config)
        return connection
    def mutate(self, config):
        # Mutate only weights
        a = self._gene_attributes[0]
        v = getattr(self, a.name)
        setattr(self, a.name, a.mutate_value(v, config))

    def distance(self, other, config):
        d = abs(self.weight - other.weight)
        if self.enabled != other.enabled:
            d += 1.0
        return d * config.compatibility_weight_coefficient
    
class FixedStructureGenomeConfig(object):
    __params = [ConfigParameter('feed_forward', bool),
                ConfigParameter('num_inputs', int),
                ConfigParameter('num_outputs', int),
                ConfigParameter('num_hidden', int),
                ConfigParameter('num_hidden_layers', int), 
                ConfigParameter('initial_connection', str, 'full_direct'),
                ConfigParameter('conn_add_prob', float),
                ConfigParameter('conn_delete_prob', float),
                ConfigParameter('node_add_prob', float),
                ConfigParameter('node_delete_prob', float),
                ConfigParameter('compatibility_disjoint_coefficient', float),
                ConfigParameter('compatibility_weight_coefficient', float),
                ConfigParameter('structural_mutation_surer', str, 'default'), 
                ConfigParameter('single_structural_mutation', bool, 'false')]


    def __init__(self, params):
        # Create full set of available activation functions.
        self.activation_defs = ActivationFunctionSet()
        self.activation_options = params.get('activation_options', 'clamped').strip().split()
        self.aggregation_options = params.get('aggregation_options', 'sum').strip().split()
        self.aggregation_function_defs = AggregationFunctionSet()

        # Gather configuration data from the gene classes.
        self.node_gene_type = params['node_gene_type']
        self.__params += self.node_gene_type.get_config_params()
        self.connection_gene_type = params['connection_gene_type']
        self.__params += self.connection_gene_type.get_config_params()
        self.connection_fraction = None
        self.node_indexer = None
        self.structural_mutation_surer = 'false'

        # Use the configuration data to interpret the supplied parameters.
        for p in self.__params:
            setattr(self, p.name, p.interpret(params))

        # By convention, input pins have negative keys, and the output
        # pins have keys 0,1,...
        self.input_keys = [-i - 1 for i in range(self.num_inputs)]
        self.output_keys = [i for i in range(self.num_outputs)]

    def get_new_node_key(self, node_dict):
        if self.node_indexer is None:
            if node_dict:
                self.node_indexer = count(max(list(node_dict)) + 1)
            else:
                self.node_indexer = count(max(list(node_dict)) + 1)

        new_id = next(self.node_indexer)

        assert new_id not in node_dict

        return new_id
    
    def save(self, f):
        write_pretty_params(f, self, self.__params)

    def check_structural_mutation_surer(self):
        if self.structural_mutation_surer == 'true':
            return True
        elif self.structural_mutation_surer == 'false':
            return False
        elif self.structural_mutation_surer == 'default':
            return self.single_structural_mutation
        else:
            error_string = f"Invalid structural_mutation_surer {self.structural_mutation_surer!r}"
            raise RuntimeError(error_string)
        
class FixedStructureGenome(DefaultGenome):
    @classmethod
    def parse_config(cls, param_dict):
        param_dict['node_gene_type'] = FixedStructureNodeGene
        param_dict['connection_gene_type'] = FixedStructureConnectionGene
        return FixedStructureGenomeConfig(param_dict)
    
    @classmethod
    def write_config(cls, f, config):
        config.save(f)

    def __init__(self, key):
        self.key = key
        self.connections = {}
        self.nodes = {}

        # Fitness results.
        self.fitness = None

    def mutate(self, config):
        """ Mutates this genome. """

        if config.single_structural_mutation:
            div = max(1, (config.node_add_prob + config.node_delete_prob +
                            config.conn_add_prob + config.conn_delete_prob))
            r = random()
            if r < (config.node_add_prob / div):
                self.mutate_add_node(config)
            elif r < ((config.node_add_prob + config.node_delete_prob) / div):
                self.mutate_delete_node(config)
            elif r < ((config.node_add_prob + config.node_delete_prob +
                        config.conn_add_prob) / div):
                self.mutate_add_connection(config)
            elif r < ((config.node_add_prob + config.node_delete_prob +
                        config.conn_add_prob + config.conn_delete_prob) / div):
                self.mutate_delete_connection()
        else:
            if random() < config.node_add_prob:
                self.mutate_add_node(config)

            if random() < config.node_delete_prob:
                self.mutate_delete_node(config)

            if random() < config.conn_add_prob:
                self.mutate_add_connection(config)

            if random() < config.conn_delete_prob:
                self.mutate_delete_connection()

        # Mutate connection genes.
        for cg in self.connections.values():
            cg.mutate(config)

        # Mutate node genes (bias, response, etc.).
        for ng in self.nodes.values():
            ng.mutate(config)

    def configure_crossover(self, genome1, genome2, config):
        """ Configure a new genome by crossover from two parent genomes. """
        if genome1.fitness > genome2.fitness:
            parent1, parent2 = genome1, genome2
        else:
            parent1, parent2 = genome2, genome1

        # Inherit connection genes
        for key, cg1 in parent1.connections.items():
            cg2 = parent2.connections.get(key)
            if cg2 is None:
                # Excess or disjoint gene: copy from the fittest parent.
                self.connections[key] = cg1.copy()
            else:
                # Homologous gene: combine genes from both parents.
                self.connections[key] = cg1.crossover(cg2)
            self.connections[key].enabled = True

        # Inherit node genes
        parent1_set = parent1.nodes
        parent2_set = parent2.nodes

        for key, ng1 in parent1_set.items():
            ng2 = parent2_set.get(key)
            assert key not in self.nodes
            if ng2 is None:
                # Extra gene: copy from the fittest parent
                self.nodes[key] = ng1.copy()
            else:
                # Homologous gene: combine genes from both parents.
                self.nodes[key] = ng1.crossover(ng2)

    def distance(self, other, config):
        """
        Returns the genetic distance between this genome and the other. This distance value
        is used to compute genome compatibility for speciation.
        """

        # Compute node gene distance component.
        node_distance = 0.0
        if self.nodes or other.nodes:
            disjoint_nodes = 0
            for k2 in other.nodes:
                if k2 not in self.nodes:
                    disjoint_nodes += 1

            for k1, n1 in self.nodes.items():
                n2 = other.nodes.get(k1)
                if n2 is None:
                    disjoint_nodes += 1
                else:
                    # Homologous genes compute their own distance value.
                    node_distance += n1.distance(n2, config)

            max_nodes = max(len(self.nodes), len(other.nodes))
            node_distance = (node_distance +
                                (config.compatibility_disjoint_coefficient *
                                disjoint_nodes)) / max_nodes

        # Compute connection gene differences.
        connection_distance = 0.0
        if self.connections or other.connections:
            disjoint_connections = 0
            for k2 in other.connections:
                if k2 not in self.connections:
                    disjoint_connections += 1

            for k1, c1 in self.connections.items():
                c2 = other.connections.get(k1)
                if c2 is None:
                    disjoint_connections += 1
                else:
                    # Homologous genes compute their own distance value.
                    connection_distance += c1.distance(c2, config)

            max_conn = max(len(self.connections), len(other.connections))
            connection_distance = (connection_distance +
                                    (config.compatibility_disjoint_coefficient *
                                    disjoint_connections)) / max_conn

        distance = node_distance + connection_distance
        return distance
    
    def size(self):
        """Returns genome 'complexity', taken to be (number of nodes, number of enabled connections)"""
        num_enabled_connections = sum([1 for cg in self.connections.values() if cg.enabled is True])
        return len(self.nodes), num_enabled_connections

    def __str__(self):
        s = f"Key: {self.key}\nFitness: {self.fitness}\nNodes:"
        for k, ng in self.nodes.items():
            s += f"\n\t{k} {ng!s}"
        s += "\nConnections:"
        connections = list(self.connections.values())
        connections.sort()
        for c in connections:
            s += "\n\t" + str(c)
        return s

    def configure_new(self, config):
        # Create node genes for the input and output nodes
        for node_key in config.output_keys:
            self.nodes[node_key] = self.create_node(config, node_key)

        # Initialize list to keep track of nodes in the last layer (start with input layer)
        last_layer_nodes = config.input_keys

        # Add hidden nodes and layers
        # config.num_hidden now represents the number of neurons PER HIDDEN LAYER
        if config.num_hidden > 0 and config.num_hidden_layers > 0:
            for _ in range(config.num_hidden_layers):
                layer_node_ids = []
                for _ in range(config.num_hidden):  # Create the specified number of nodes for this layer
                    node_key = config.get_new_node_key(self.nodes)
                    self.nodes[node_key] = self.create_node(config, node_key)
                    layer_node_ids.append(node_key)
                
                # Connect all nodes from the last layer to the current layer's nodes
                for last_layer_node in last_layer_nodes:
                    for current_layer_node in layer_node_ids:
                        connection = self.create_connection(config, last_layer_node, current_layer_node)
                        self.connections[connection.key] = connection

                # Update last_layer_nodes to reflect the current layer for next iteration
                last_layer_nodes = layer_node_ids

            # Connect all nodes from the last hidden layer to the output nodes
            for hidden_node_id in last_layer_nodes:
                for output_node_id in config.output_keys:
                    connection = self.create_connection(config, hidden_node_id, output_node_id)
                    self.connections[connection.key] = connection
        else:
            # If there are no hidden layers, connect input nodes directly to output nodes
            for input_node_id in config.input_keys:
                for output_node_id in config.output_keys:
                    connection = self.create_connection(config, input_node_id, output_node_id)
                    self.connections[connection.key] = connection

    @staticmethod
    def create_node(config, node_id):
        node = FixedStructureNodeGene(node_id)
        node.init_attributes(config)
        return node

    @staticmethod
    def create_connection(config, input_id, output_id):
        connection = config.connection_gene_type((input_id, output_id))
        connection.init_attributes(config)
        return connection

In [4]:
# if r < mutate_rate:
#             mutate_power = getattr(config, self.mutate_power_name)
#             return self.clamp(value + gauss(0.0, mutate_power), config)

#         replace_rate = getattr(config, self.replace_rate_name)

#         r = random()
#         if r < replace_rate:
#             return self.init_value(config)

#         return value

#     def validate(self, config):
#         min_value = getattr(config, self.min_value_name)
#         max_value = getattr(config, self.max_value_name)
#         if max_value < min_value:
#             raise RuntimeError("Invalid min/max configuration for {self.name}")

# class FixedStructureNodeGene(BaseGene):
#     _gene_attributes = [
#                         FloatAttribute('bias'),
#                         FloatAttribute('response'),
#                         StringAttribute('activation', options=''),
#                         StringAttribute('aggregation', options='')]

#     def distance(self, other, config):
#         d = abs(self.bias - other.bias) + abs(self.response - other.response)
#         if self.activation != other.activation:
#             d += 1.0
#         if self.aggregation != other.aggregation:
#             d += 1.0
#         return d * config.compatibility_weight_coefficient

# class FixedStructureConnectionGene(BaseGene):
#     _gene_attributes = [FixedFloatAttribute('weight'),
#                         BoolAttribute('enabled')]

#     def mutate(self, config):
#         # Mutate only weights
#         a = self._gene_attributes[0]
#         v = getattr(self, a.name)
#         setattr(self, a.name, a.mutate_value(v, config))

#     def distance(self, other, config):
#         d = abs(self.weight - other.weight)
#         if self.enabled != other.enabled:
#             d += 1.0
#         return d * config.compatibility_weight_coefficient
    
# class FixedStructureGenomeConfig(object):
#     __params = [ConfigParameter('feed_forward', bool),
#                 ConfigParameter('num_inputs', int),
#                 ConfigParameter('num_outputs', int),
#                 ConfigParameter('num_hidden', int),
#                 ConfigParameter('num_hidden_layers', int), 
#                 ConfigParameter('initial_connection', str, 'full_direct'),
#                 ConfigParameter('conn_add_prob', float),
#                 ConfigParameter('conn_delete_prob', float),
#                 ConfigParameter('node_add_prob', float),
#                 ConfigParameter('node_delete_prob', float),
#                 ConfigParameter('compatibility_disjoint_coefficient', float),
#                 ConfigParameter('compatibility_weight_coefficient', float),
#                 ConfigParameter('structural_mutation_surer', str, 'default'), 
#                 ConfigParameter('single_structural_mutation', bool, 'false')]


#     def __init__(self, params):
#         # Create full set of available activation functions.
#         self.activation_defs = ActivationFunctionSet()
#         self.activation_options = params.get('activation_options', 'clamped').strip().split()
#         self.aggregation_options = params.get('aggregation_options', 'sum').strip().split()
#         self.aggregation_function_defs = AggregationFunctionSet()

#         # Gather configuration data from the gene classes.
#         self.node_gene_type = params['node_gene_type']
#         self.__params += self.node_gene_type.get_config_params()
#         self.connection_gene_type = params['connection_gene_type']
#         self.__params += self.connection_gene_type.get_config_params()
#         self.connection_fraction = None
#         self.node_indexer = None
#         self.structural_mutation_surer = 'false'

#         # Use the configuration data to interpret the supplied parameters.
#         for p in self.__params:
#             setattr(self, p.name, p.interpret(params))

#         # By convention, input pins have negative keys, and the output
#         # pins have keys 0,1,...
#         self.input_keys = [-i - 1 for i in range(self.num_inputs)]
#         self.output_keys = [i for i in range(self.num_outputs)]

#     def get_new_node_key(self, node_dict):
#         if self.node_indexer is None:
#             if node_dict:
#                 self.node_indexer = count(max(list(node_dict)) + 1)
#             else:
#                 self.node_indexer = count(max(list(node_dict)) + 1)

#         new_id = next(self.node_indexer)

#         assert new_id not in node_dict

#         return new_id
    
#     def save(self, f):
#         write_pretty_params(f, self, self.__params)

#     def check_structural_mutation_surer(self):
#         if self.structural_mutation_surer == 'true':
#             return True
#         elif self.structural_mutation_surer == 'false':
#             return False
#         elif self.structural_mutation_surer == 'default':
#             return self.single_structural_mutation
#         else:
#             error_string = f"Invalid structural_mutation_surer {self.structural_mutation_surer!r}"
#             raise RuntimeError(error_string)
        
# class FixedStructureGenome(DefaultGenome):
#     @classmethod
#     def parse_config(cls, param_dict):
#         param_dict['node_gene_type'] = FixedStructureNodeGene
#         param_dict['connection_gene_type'] = FixedStructureConnectionGene
#         return FixedStructureGenomeConfig(param_dict)
    
#     @classmethod
#     def write_config(cls, f, config):
#         config.save(f)

#     def __init__(self, key):
#         self.key = key
#         self.connections = {}
#         self.nodes = {}

#         # Fitness results.
#         self.fitness = None

#     def mutate(self, config):
#         """ Mutates this genome. """

#         if config.single_structural_mutation:
#             div = max(1, (config.node_add_prob + config.node_delete_prob +
#                             config.conn_add_prob + config.conn_delete_prob))
#             r = random()
#             if r < (config.node_add_prob / div):
#                 self.mutate_add_node(config)
#             elif r < ((config.node_add_prob + config.node_delete_prob) / div):
#                 self.mutate_delete_node(config)
#             elif r < ((config.node_add_prob + config.node_delete_prob +
#                         config.conn_add_prob) / div):
#                 self.mutate_add_connection(config)
#             elif r < ((config.node_add_prob + config.node_delete_prob +
#                         config.conn_add_prob + config.conn_delete_prob) / div):
#                 self.mutate_delete_connection()
#         else:
#             if random() < config.node_add_prob:
#                 self.mutate_add_node(config)

#             if random() < config.node_delete_prob:
#                 self.mutate_delete_node(config)

#             if random() < config.conn_add_prob:
#                 self.mutate_add_connection(config)

#             if random() < config.conn_delete_prob:
#                 self.mutate_delete_connection()

#         # Mutate connection genes.
#         for cg in self.connections.values():
#             cg.mutate(config)

#         # Mutate node genes (bias, response, etc.).
#         for ng in self.nodes.values():
#             ng.mutate(config)

#     def configure_crossover(self, genome1, genome2, config):
#         """ Configure a new genome by crossover from two parent genomes. """
#         if genome1.fitness > genome2.fitness:
#             parent1, parent2 = genome1, genome2
#         else:
#             parent1, parent2 = genome2, genome1

#         # Inherit connection genes
#         for key, cg1 in parent1.connections.items():
#             cg2 = parent2.connections.get(key)
#             if cg2 is None:
#                 # Excess or disjoint gene: copy from the fittest parent.
#                 self.connections[key] = cg1.copy()
#             else:
#                 # Homologous gene: combine genes from both parents.
#                 self.connections[key] = cg1.crossover(cg2)
#             self.connections[key].enabled = True

#         # Inherit node genes
#         parent1_set = parent1.nodes
#         parent2_set = parent2.nodes

#         for key, ng1 in parent1_set.items():
#             ng2 = parent2_set.get(key)
#             assert key not in self.nodes
#             if ng2 is None:
#                 # Extra gene: copy from the fittest parent
#                 self.nodes[key] = ng1.copy()
#             else:
#                 # Homologous gene: combine genes from both parents.
#                 self.nodes[key] = ng1.crossover(ng2)

#     def distance(self, other, config):
#         """
#         Returns the genetic distance between this genome and the other. This distance value
#         is used to compute genome compatibility for speciation.
#         """

#         # Compute node gene distance component.
#         node_distance = 0.0
#         if self.nodes or other.nodes:
#             disjoint_nodes = 0
#             for k2 in other.nodes:
#                 if k2 not in self.nodes:
#                     disjoint_nodes += 1

#             for k1, n1 in self.nodes.items():
#                 n2 = other.nodes.get(k1)
#                 if n2 is None:
#                     disjoint_nodes += 1
#                 else:
#                     # Homologous genes compute their own distance value.
#                     node_distance += n1.distance(n2, config)

#             max_nodes = max(len(self.nodes), len(other.nodes))
#             node_distance = (node_distance +
#                                 (config.compatibility_disjoint_coefficient *
#                                 disjoint_nodes)) / max_nodes

#         # Compute connection gene differences.
#         connection_distance = 0.0
#         if self.connections or other.connections:
#             disjoint_connections = 0
#             for k2 in other.connections:
#                 if k2 not in self.connections:
#                     disjoint_connections += 1

#             for k1, c1 in self.connections.items():
#                 c2 = other.connections.get(k1)
#                 if c2 is None:
#                     disjoint_connections += 1
#                 else:
#                     # Homologous genes compute their own distance value.
#                     connection_distance += c1.distance(c2, config)

#             max_conn = max(len(self.connections), len(other.connections))
#             connection_distance = (connection_distance +
#                                     (config.compatibility_disjoint_coefficient *
#                                     disjoint_connections)) / max_conn

#         distance = node_distance + connection_distance
#         return distance
    
#     def size(self):
#         """Returns genome 'complexity', taken to be (number of nodes, number of enabled connections)"""
#         num_enabled_connections = sum([1 for cg in self.connections.values() if cg.enabled is True])
#         return len(self.nodes), num_enabled_connections

#     def __str__(self):
#         s = f"Key: {self.key}\nFitness: {self.fitness}\nNodes:"
#         for k, ng in self.nodes.items():
#             s += f"\n\t{k} {ng!s}"
#         s += "\nConnections:"
#         connections = list(self.connections.values())
#         connections.sort()
#         for c in connections:
#             s += "\n\t" + str(c)
#         return s

#     def configure_new(self, config):
#         # Create node genes for the input and output nodes
#         for node_key in config.output_keys:
#             self.nodes[node_key] = self.create_node(config, node_key)

#         # Initialize list to keep track of nodes in the last layer (start with input layer)
#         last_layer_nodes = config.input_keys

#         # Add hidden nodes and layers
#         # config.num_hidden now represents the number of neurons PER HIDDEN LAYER
#         if config.num_hidden > 0 and config.num_hidden_layers > 0:
#             for _ in range(config.num_hidden_layers):
#                 layer_node_ids = []
#                 for _ in range(config.num_hidden):  # Create the specified number of nodes for this layer
#                     node_key = config.get_new_node_key(self.nodes)
#                     self.nodes[node_key] = self.create_node(config, node_key)
#                     layer_node_ids.append(node_key)
                
#                 # Connect all nodes from the last layer to the current layer's nodes
#                 for last_layer_node in last_layer_nodes:
#                     for current_layer_node in layer_node_ids:
#                         connection = self.create_connection(config, last_layer_node, current_layer_node)
#                         self.connections[connection.key] = connection

#                 # Update last_layer_nodes to reflect the current layer for next iteration
#                 last_layer_nodes = layer_node_ids

#             # Connect all nodes from the last hidden layer to the output nodes
#             for hidden_node_id in last_layer_nodes:
#                 for output_node_id in config.output_keys:
#                     connection = self.create_connection(config, hidden_node_id, output_node_id)
#                     self.connections[connection.key] = connection
#         else:
#             # If there are no hidden layers, connect input nodes directly to output nodes
#             for input_node_id in config.input_keys:
#                 for output_node_id in config.output_keys:
#                     connection = self.create_connection(config, input_node_id, output_node_id)
#                     self.connections[connection.key] = connection

#     @staticmethod
#     def create_node(config, node_id):
#         node = FixedStructureNodeGene(node_id)
#         node.init_attributes(config)
#         return node

#     @staticmethod
#     def create_connection(config, input_id, output_id):
#         connection = config.connection_gene_type((input_id, output_id))
#         connection.init_attributes(config)
#         return connection

In [5]:
NUM_RUNS = 5
MAX_GENS = 2000
generation = 0
SAVE_INTERVAL = 50

In [6]:
engine_config_channel = EngineConfigurationChannel()
engine_config_channel.set_configuration_parameters(time_scale=10)
agent_count_channel = AgentLogChannel()

env_path = "../Builds/train-env/autonomous-drone.exe"
save_nn_destination = 'result/best.pkl'

env = UnityEnvironment(file_name=None, worker_id=0, no_graphics=True, side_channels=[engine_config_channel, agent_count_channel])
env.reset()

In [7]:
out_mult = 1

behavior_specs = env.behavior_specs
print(f"Behaviour specs {behavior_specs}")
behavior_name = list(behavior_specs)[0]
spec = env.behavior_specs[behavior_name]

print(f"Name of the behavior : {behavior_name}")
print("Number of observations : ", len(spec.observation_specs)) # vector if 1

# Is the Action continuous or multi-discrete ?
if spec.action_spec.continuous_size > 0:
  print(f"There are {spec.action_spec.continuous_size} continuous actions")
if spec.action_spec.is_discrete():
  print(f"There are {spec.action_spec.discrete_size} discrete actions")

Behaviour specs <mlagents_envs.base_env.BehaviorMapping object at 0x0000027D0AF86740>
Name of the behavior : ForestControl?team=0
Number of observations :  4
There are 4 continuous actions


In [8]:
def create_folder(file_name_prefix):
    directory = os.path.dirname(f"{file_name_prefix}")
    if not os.path.exists(directory):
        os.makedirs(directory)

In [9]:
def set_agents_and_double_reset(num_agents: int):
    agent_count_channel.send_int(data=num_agents) 
    env.reset()
    env.reset()

In [10]:
def get_observation_for_agent(agent: int, observations):
    for observation in observations:
        key = int(observation.split("=")[2])
        if key == agent:
            return observations[observation]

In [11]:
def map_agent_ids(decision_steps):
    """
    Map agent ids between NEAT and UNITY.

    Args:
        decision_steps: An iterable containing decision steps.

    Returns:
        A tuple of two dictionaries: (unity_to_neat_map, neat_to_unity_map)
    """
    unity_to_neat_map = {}
    neat_to_unity_map = {}
    id_count = 0
    for step in decision_steps:
        unity_to_neat_map[step] = id_count
        neat_to_unity_map[id_count] = step
        id_count += 1
    return unity_to_neat_map, neat_to_unity_map

In [12]:

def create_policies(genomes, cfg):
    policies = []
    for _, g in genomes:
        g.fitness = 0
        policy = neat.nn.FeedForwardNetwork.create(g, cfg)
        policies.append(policy)
    return policies

In [13]:
def eval_genomes(genomes, cfg):
    global generation 
    generation += 1
    policies = create_policies(genomes, cfg)
    set_agents_and_double_reset(len(policies))
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    agent_count = len(decision_steps.agent_id)
    unity_to_neat_map, neat_to_unity_map = map_agent_ids(decision_steps)
    episode_rewards = [0] * agent_count
    print(f"Agent count: {agent_count}")
    done = False  # Vectorized initialization
    removed_agents = []
    while not done:
        for agent in decision_steps:
            if unity_to_neat_map[agent] not in removed_agents:
                obs = np.concatenate(decision_steps[agent].obs[:])
                nn_input =  np.asarray(obs)
                actions = policies[unity_to_neat_map[agent]].activate(nn_input)
                #print(f"NN OUTPUT: {actions}, agent: {agent}")
                continous_actions = np.asarray([actions])
                continous_actions = np.clip(continous_actions, -1, 1)
                action_tuple = ActionTuple(discrete=None, continuous=continous_actions)
                env.set_action_for_agent(behavior_name=behavior_name, 
                                        agent_id=agent, 
                                        action=action_tuple)
        env.step()
        decision_steps, terminal_steps = env.get_steps(behavior_name)
        for agent in range(agent_count):
            if agent not in removed_agents:
                local_agent = neat_to_unity_map[agent]
                if local_agent in terminal_steps:
                    episode_rewards[agent] += terminal_steps[local_agent].reward
                    removed_agents.append(agent)
                    #print(f"Finished: {agent}")
                elif local_agent in decision_steps:
                    episode_rewards[agent] += decision_steps[local_agent].reward
        if len(removed_agents) >= agent_count:
            done = True
    for i, (_, genome) in enumerate(genomes):
        genome.fitness = episode_rewards[i]
    if generation % SAVE_INTERVAL == 0:
      global file_name_prefix
      best_genome_current_generation = max(genomes, key=lambda x: x[1].fitness) 
      with open(file_name_prefix+'best-'+str(generation)+'.pkl', 'wb') as f:
          pickle.dump(best_genome_current_generation, f)
      

In [14]:
def run(config_file, run, datte):
    print(f"Running {run}")
    global file_name_prefix
    file_name_prefix = f"forest-env-fixed-nn/checkpoints/{datte}/run-{run}/"
    create_folder(file_name_prefix=file_name_prefix)

    config = neat.Config(FixedStructureGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)
    
    pop = neat.Population(config)

    stats = neat.StatisticsReporter()

    pop.add_reporter(stats)
    #pop.add_reporter(neat.Checkpointer(generation_interval=5, time_interval_seconds=100000, filename_prefix=file_name_prefix))
    pop.add_reporter(neat.TBReporter(True, 0, run, datte))
    #pop.add_reporter(neat.StdOutReporter(True))
    best = pop.run(eval_genomes, MAX_GENS)
    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(best))
    print("Finished running!")
    
    # Save best genome
    with open(f'logs/{datte}/{run}/best.pkl', 'wb') as f:
        pickle.dump(best, f)

In [15]:
config_path = '../configs/config_more_layers_forest'
datte = datetime.datetime.now().strftime("%d-%m-%Y--%H_%M") 
# pre create folders for checkpointer
for r in range(NUM_RUNS):
    run(config_path, r, datte)

Running 0

 ****** Running generation 0 ****** 

Agent count: 100
Population's average fitness: -9.99653 stdev: 0.03424
Best fitness: -9.65585 - size: (6, 124) - species 89 - id 89
Best fitness ever found! : -9.655849605156504
Population of 218 members in 100 species:
   ID   age  size   fitness   adj fit  stag
  ====  ===  ====  =========  =======  ====
     1    0     2      -10.0      0.0     0
     2    0     2  -9.999964748723869  3.525127613102086e-05     0
     3    0     2      -10.0      0.0     0
     4    0     2      -10.0      0.0     0
     5    0     2      -10.0      0.0     0
     6    0     2      -10.0      0.0     0
     7    0     2      -10.0      0.0     0
     8    0     2      -10.0      0.0     0
     9    0     2  -9.999938897961968  6.110203803189052e-05     0
    10    0     2      -10.0      0.0     0
    11    0     2      -10.0      0.0     0
    12    0     2      -10.0      0.0     0
    13    0     2      -10.0      0.0     0
    14    0     2      -1

In [ ]:
env.close()

In [ ]:
import visualize
import pickle
save_nn_destination = '../result/best.pkl'
config_path = '../configs/config_more_layers_forest'

with open(save_nn_destination, "rb") as f:
    genome = pickle.load(f)
    genome = genome[1]
    print('\nBest genome:\n{!s}'.format(genome))
print(genome.fitness)
config = neat.Config(FixedStructureGenome, neat.DefaultReproduction,
                    neat.DefaultSpeciesSet, neat.DefaultStagnation,
                    config_path)
visualize.draw_net(config, genome, True)